In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name='quick-starts-ws-142074',
                   subscription_id='6b4af8be-9931-443e-90f6-c4c34a1f9737',
                   resource_group='aml-quickstarts-142074')
exp = Experiment(workspace=ws, name="hyperdrive")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-142074
Azure region: southcentralus
Subscription id: 6b4af8be-9931-443e-90f6-c4c34a1f9737
Resource group: aml-quickstarts-142074


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            min_nodes=0,
                                                            max_nodes=4, 
                                                            idle_seconds_before_scaledown=2400)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

Writing conda_dependencies.yml


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import Environment, ScriptRunConfig
from shutil import copyfile
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    'C' : uniform(0.001, 2.0),
    'max_iter' : choice(50, 100, 150, 200, 250)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.01)

if "training" not in os.listdir():
    os.mkdir("./training")
    
copyfile('./train.py', './training/train.py')

## !!!DEPRECATED!!! ##
# # Create a SKLearn estimator for use with train.py
# est = SKLearn(source_directory = ".",
#               compute_target=cpu_cluster,
#               entry_script='train.py')
########################

# Create SKLearn environment
sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

src = ScriptRunConfig(source_directory='./training/',
                      script='train.py',
                      arguments=['--C', 1.0, '--max_iter', 100],
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, 
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=50,
                                     max_concurrent_runs=4)

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [8]:
import joblib
from azureml.core import Model
# Get your best run and save the model from that run.

best_run = hyperdive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_parameters = best_run.get_details()['runDefinition']['arguments']

print('\nBest run ID: ', best_run.id)
print('\nAccuracy: ', best_run_metrics['Accuracy'])
print('\nInverse Regularization C: ', best_run_parameters[5])
print('\nMax Iterations: ', best_run_parameters[7])

hd_model = best_run.register_model(model_name = 'sklearn-logreg-hd.joblib',
                                   model_path = './outputs/hd-model.joblib',
                                   model_framework = Model.Framework.SCIKITLEARN,
                                   model_framework_version = '0.19.1'
)


Best run ID:  HD_435f24d7-f05f-4135-90c0-df4949696aaf_18

Accuracy:  0.9059237679048313

Inverse Regularization C:  0.07620816694019843

Max Iterations:  250


In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [10]:
import pandas as pd
from train import clean_data
from azureml.core import Datastore

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

# Register dataset for use in automl job
datastore = Datastore.get(ws, 'workspaceblobstore')
data = TabularDatasetFactory.register_pandas_dataframe(pd.concat([x, y], axis=1),
                                                       datastore,
                                                       'bank-marketing-clean',
                                                       description='Cleaned bank marketing dataset from automlsamplenotebookdata.',
                                                       tags=None,
                                                       show_progress=True)




Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/681acb20-a8eb-4e9b-a99f-a5ffda170f36/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [11]:
# Split data between training and test sets
train_data, test_data = data.random_split(0.75, seed=12)

In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=cpu_cluster)

In [13]:
# Submit your automl run

exp = Experiment(workspace=ws, name="auto-ml")
run = exp.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_9e056b26-4d93-479f-888c-bd0afe4b4532

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a fal

In [14]:
# Retrieve and save your best automl model.

best_run, fitted_model = run.get_output()

print(best_run)
print(fitted_model)

joblib.dump(fitted_model, './outputs/automl-model.joblib')

ConfigException: ConfigException:
	Message: The model you attempted to retrieve requires 'xgboost' to be installed at '==0.90'. You have 'xgboost==1.3.3', please reinstall 'xgboost==0.90' (e.g. `pip install xgboost==0.90`) and rerun the previous command.
	InnerException: XGBoostError: [10:40:12] /mnt/xgboost/src/learner.cc:922: Check failed: header == serialisation_header_: 

  If you are loading a serialized model (like pickle in Python) generated by older
  XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version.  There's a simple script for helping
  the process. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for reference to the script, and more details about differences between saving model and
  serializing.


Stack trace:
  [bt] (0) /anaconda/envs/azureml_py36/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x7c) [0x7fbbbc85464c]
  [bt] (1) /anaconda/envs/azure

In [15]:
# Delete compute cluster
cpu_cluster.delete()